In [30]:
# basic dependencies

import numpy as np
from numpy import loadtxt
from numpy import savetxt

import pandas as pd
import math
import time
import joblib

np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

###########

# torch dependencies
import torch

tkwargs = {"dtype": torch.double, # set as double to minimize zero error for cholesky decomposition error
           "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu")} # set tensors to GPU, if multiple GPUs please set cuda:x properly
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.set_printoptions(precision=3)

###########

# plotting dependencies
import matplotlib
from matplotlib import pyplot as plt
from matplotlib.cm import ScalarMappable

%matplotlib inline

# sk dependencies
import sklearn

# data related
from botorch.utils.sampling import draw_sobol_samples
from botorch.utils.transforms import unnormalize, normalize

# surrogate model specific
from botorch.models.gp_regression import SingleTaskGP
from botorch.models.model_list_gp_regression import ModelListGP
from botorch.models.transforms.outcome import Standardize
from gpytorch.mlls.sum_marginal_log_likelihood import SumMarginalLogLikelihood
from botorch import fit_gpytorch_model

# qNEHVI specific
from botorch.acquisition.multi_objective.objective import IdentityMCMultiOutputObjective
from botorch.acquisition.multi_objective.monte_carlo import qNoisyExpectedHypervolumeImprovement

# utilities
from botorch.optim.optimize import optimize_acqf
from botorch.sampling.samplers import SobolQMCNormalSampler
from botorch.utils.multi_objective.pareto import is_non_dominated
from botorch.utils.multi_objective.hypervolume import Hypervolume
from typing import Optional
from torch import Tensor
from botorch.exceptions import BadInitialCandidatesWarning

import warnings

warnings.filterwarnings('ignore', category=BadInitialCandidatesWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)

# pymoo dependencies
import pymoo

from pymoo.util.ref_dirs import get_reference_directions
from pymoo.core.problem import ElementwiseProblem, Problem

from pymoo.algorithms.moo.unsga3 import UNSGA3
from pymoo.optimize import minimize
from pymoo.core.termination import NoTermination

from sklearn.metrics import r2_score, mean_squared_error

In [8]:
def func_train_model (train_x_gp, train_obj):
    
    # define and train surrogate models for objective and constraint
    models = []

    # models for objective

    for i in range(train_obj.shape[-1]):
        models.append(SingleTaskGP(train_x_gp, train_obj[..., i : i + 1], outcome_transform=Standardize(m=1)))

    model = ModelListGP(*models)
    mll = SumMarginalLogLikelihood(model.likelihood, model)

    fit_gpytorch_model(mll) 

    acq_func = qNoisyExpectedHypervolumeImprovement(
        model=model,
        ref_point=ref_point, # for computing HV, must flip for BoTorch
        X_baseline=train_x_gp, # feed total list of train_x for this current iteration
        sampler=SobolQMCNormalSampler(num_samples=128),  # determines how candidates are randomly proposed before selection
        objective=IdentityMCMultiOutputObjective(outcomes=np.arange(n_obj).tolist()), # optimize first n_obj col 
        #constraints=create_idxrs(), # constraint on last n_constr col
        prune_baseline=True, cache_pending=True)  # options for improving qNEHVI, keep these on
    
    return model, acq_func

def func_model_accuracy(train_x_gp, train_obj, model):
    
    with torch.no_grad():
        # just the training set
        posterior = model.posterior(train_x_gp)
        y_pred = posterior.mean
        lower, upper = posterior.mvn.confidence_region()
        error = upper-lower

    fig, ax = plt.subplots(ncols = train_obj.shape[1], figsize = (train_obj.shape[1]*4,4))

    # plot objective predictions
    for i in np.arange(train_obj.shape[1]).tolist():

        score1 = r2_score(train_obj[:,i].cpu().numpy(), y_pred[:,i].cpu().numpy())
        score2 = mean_squared_error(train_obj[:,i].cpu().numpy(), y_pred[:,i].cpu().numpy())

        ax[i].axline((1, 1), slope=1)

        ax[i].errorbar(x=train_obj[:,i].cpu().numpy(), y=y_pred[:,i].cpu().numpy(), yerr=error[:,i].cpu().numpy(),
                       ls='', marker='D', mec='w', mew=0.2, mfc='b', c='b', alpha=0.35,
                        label='Trained Points'
                       )

        ax[i].set_title(f"f{i}", fontsize=18)
        ax[i].set_xlabel('True Value', fontsize=14)

        textstr = 'R2-score:'+str('%.7f'%score1)+'\n'+'MSE-score:'+str('%.4E'%score2)
        props = dict(boxstyle='square', facecolor='white', alpha=0.5)
        # place a text box in upper left in axes coords
        ax[i].text(0.05, 0.95, textstr, transform=ax[i].transAxes, fontsize=11,
                verticalalignment='top', bbox=props)

    ax[0].set_ylabel('Predicted Value', fontsize=14)

def func_repair (candidates): 
    
    # repair inputs to feasibility
    for i in range(candidates.shape[0]):
        if 0.3 - candidates[i,1]/candidates[i,4] > 0:
            candidates[i,4] = min(candidates[i,1]/0.3, candidates[i,4])

    for i in range(candidates.shape[0]):
        if 2 - (candidates[i,1]/candidates[i,4]) - (candidates[i,3]/candidates[i,1]) > 0:
            candidates[i,3] = max((2-candidates[i,1]/candidates[i,4])*candidates[i,1], candidates[i,3])
    return candidates

def func_unsga3 (train_x_gp, train_obj):
    
    # we pick out the best points so far to form parents
    pareto_mask = is_non_dominated(train_obj)
    pareto_y = -train_obj[pareto_mask]
    pareto_x = train_x_gp[pareto_mask]

    algorithm = UNSGA3(pop_size=256,
                       ref_dirs=get_reference_directions("energy", n_obj, BATCH_SIZE, seed=random_state),
                       sampling=pareto_x.cpu().numpy(),
                      )

    pymooproblem = Problem(n_var=n_var, n_obj=n_obj, xl=np.zeros(n_var), xu=np.ones(n_var))
    algorithm.setup(pymooproblem, termination=NoTermination())

    # set the 1st population to the current evaluated population
    pop = algorithm.ask()
    pop.set("F", pareto_y.cpu().numpy())
    algorithm.tell(infills=pop)

    # propose children based on tournament selection -> crossover/mutation
    newpop = algorithm.ask()
    nsga3_x = torch.tensor(newpop.get("X"), **tkwargs)
    nsga3_x = unnormalize(nsga3_x, bounds=problem_bounds)
    
    return nsga3_x

def func_optimization (candidates, model):

    acq_list, pred_list, uncertainty_list = [], [], []
    for i in range(0, candidates.shape[0]):
        with torch.no_grad():
            acq_value = acq_func(candidates[i].unsqueeze(dim=0))
            acq_list.append(acq_value.item())

            posterior = model.posterior(candidates[i].unsqueeze(0))
            pred_list.append(posterior.mean[0].cpu().numpy())
            lower, upper = posterior.mvn.confidence_region()
            uncertainty_list.append(upper[0].cpu().numpy() - lower[0].cpu().numpy())

    sorted_x = candidates.cpu().numpy()[np.argsort((acq_list))]
    new_x = torch.tensor(sorted_x[-BATCH_SIZE:], **tkwargs)  # take best BATCH_SIZE samples from sorted pool

    return new_x, acq_list, pred_list, uncertainty_list

def func_plot_monitoring (acq_list, pred_list, uncertainty_list):
    
    acq_plot = np.array(acq_list)
    pred_plot = np.array(pred_list)
    uncertainty_plot = np.array(uncertainty_list)

    views = [(1, (0, -45, 0)),
             (2, (0, 45, 0)),
             (3, (0, 135, 0)),]
    layout = [[1, 2, 3]]  

    norm = plt.Normalize(acq_plot.min(), acq_plot.max())
    fig, ax = plt.subplot_mosaic(layout, subplot_kw={'projection': '3d'},
                                  figsize=(21, 7))

    for plane, angles in views:

        ax[plane].scatter3D(pred_plot[:,0], pred_plot[:,1], pred_plot[:,2],
                            c=acq_plot, s=20, norm=norm, cmap='viridis', alpha=0.15)

        high = pred_plot[np.argsort((acq_plot))][-BATCH_SIZE:]
        ax[plane].scatter3D(high[:,0], high[:,1], high[:,2],
                            c='y', s=100, marker='*', alpha=0.9)

        ax[plane].set_xlabel('f1', fontsize=11)
        ax[plane].set_ylabel('f2', fontsize=11)
        ax[plane].set_zlabel('f3', fontsize=11)

        ax[plane].set_xlim(initial_x_pandas['y1'].min(), initial_x_pandas['y1'].max())
        ax[plane].set_ylim(initial_x_pandas['y2'].min(), initial_x_pandas['y2'].max())
        ax[plane].set_zlim(initial_x_pandas['y3'].min(), initial_x_pandas['y3'].max())

        ax[plane].view_init(azim=angles[1])

    ax[2].set_title(f"Predicted Candidates wrt to AcqValue, proposed batch in yellow", fontsize=18)

    norm = plt.Normalize(uncertainty_plot.sum(axis=1).min(), uncertainty_plot.sum(axis=1).max())
    fig, ax = plt.subplot_mosaic(layout, subplot_kw={'projection': '3d'},
                                  figsize=(21, 7))

    for plane, angles in views:

        ax[plane].scatter3D(pred_plot[:,0], pred_plot[:,1], pred_plot[:,2],
                            c=uncertainty_plot.sum(axis=1), s=20, norm=norm, cmap='Blues', alpha=0.15)

        high = pred_plot[np.argsort((acq_plot))][-BATCH_SIZE:]
        ax[plane].scatter3D(high[:,0], high[:,1], high[:,2],
                            c='y', s=100, marker='*', alpha=0.9)

        ax[plane].set_xlabel('f1', fontsize=11)
        ax[plane].set_ylabel('f2', fontsize=11)
        ax[plane].set_zlabel('f3', fontsize=11)

        ax[plane].set_xlim(initial_x_pandas['y1'].min(), initial_x_pandas['y1'].max())
        ax[plane].set_ylim(initial_x_pandas['y2'].min(), initial_x_pandas['y2'].max())
        ax[plane].set_zlim(initial_x_pandas['y3'].min(), initial_x_pandas['y3'].max())

        ax[plane].view_init(azim=angles[1])

    ax[2].set_title(f"Predicted Candidates wrt to Uncertainty, proposed batch in yellow", fontsize=18)

In [3]:
########## settings to change every day

total_exp = 16 # total sets of experiments to run today

algo = 'pure' # start from pure
BATCH_SIZE = 4

########## settings to keep, used for some parts of the code

exp_counter = 15 # current number of experiments done, start with 0 for initialization

n_var = 5
n_obj = 3

random_state = 1
torch.manual_seed(random_state) # gives a consistent seed based on the trial number

ref_point = torch.tensor([0, 0, 0], **tkwargs)
hv=Hypervolume(ref_point=ref_point)

problem_bounds = torch.zeros(2, n_var, **tkwargs)
problem_bounds[0] = 0.6
problem_bounds[1] = 24.0

standard_bounds = torch.zeros(2, n_var, **tkwargs)
standard_bounds[1] = 1

initial_sample_size = 12 # no of initialized LHS samples

In [ ]:
while exp_counter < total_exp+1:
    t0 = time.time()
     
    ##########
    # load data from matlab/labview into jupyter       
    if algo == 'pure':
        input_file = "Data Analysis/Algo1/Results_Algo1_Run" + str(exp_counter) + ".csv"
        if Path(input_file).is_file() == False:
            print("No input file detected, sleeping for 30 sec")
            time.sleep(30)
            continue;
            
        initial_x_pandas = pd.read_csv(f"{input_file}", delimiter=',') 
        initial_x_torch = torch.tensor(initial_x_pandas.iloc[:,1:].values, **tkwargs) 
        
    else:
        input_file = "Data Analysis/Algo2/Results_Algo2_Run" + str(exp_counter) + ".csv"
        if Path(input_file).is_file() == False:
            print("No input file detected, sleeping for 30 sec")
            time.sleep(30)
            continue;
            
        initial_x_pandas = pd.read_csv(f"{input_file}", delimiter=',') 
        initial_x_torch = torch.tensor(initial_x_pandas.iloc[:,1:].values, **tkwargs) 
        
    print("File successfully found! Proceeding with optimization.")

    ####################
    # initial training data
    train_x = initial_x_torch[:,:n_var]
    train_x_gp = normalize(train_x, problem_bounds)
    train_obj = initial_x_torch[:,n_var:n_var+n_obj]

    # surrogate model
    model, acq_func = func_train_model(train_x_gp, train_obj)
    model.eval()
    func_model_accuracy(train_x_gp, train_obj, model)

    ####################    
    # optimization

    if algo == 'pure':
        
        print("Performing pure BO.")

        # taking 256 x 2 QMC candidates
        sobol_x1 = draw_sobol_samples(bounds=problem_bounds,n=256, q=1).squeeze(1)
        sobol_x2 = draw_sobol_samples(bounds=problem_bounds,n=256, q=1).squeeze(1)
        candidates = torch.cat([sobol_x1, sobol_x2])
        candidates = func_repair(candidates)
        candidates = normalize(candidates, problem_bounds)
        
        new_x, acq_list, pred_list, uncertainty_list = func_optimization(candidates, model)

        func_plot_monitoring(acq_list, pred_list, uncertainty_list)

    else:
        
        print("Performing hybrid BO+EA.")
        
        # taking 256 x 1 QMC candidates and 256 x 1 evolution candidates
        sobol_x = draw_sobol_samples(bounds=problem_bounds,n=256, q=1).squeeze(1)
        nsga3_x = func_unsga3(train_x_gp, train_obj)
        candidates = torch.cat([sobol_x, nsga3_x])
        candidates = func_repair(candidates)
        candidates = normalize(candidates, problem_bounds)
        
        new_x, acq_list, pred_list, uncertainty_list = func_optimization(candidates, model)

        func_plot_monitoring(acq_list, pred_list, uncertainty_list)
        
        candidate_check = torch.cat([torch.zeros(sobol_x.shape[0]), torch.ones(nsga3_x.shape[0])])[np.argsort((acq_list))][-BATCH_SIZE:].cpu().numpy()

    ####################
    # end of optimization loop

    # unormalize our training inputs back to original problem bounds
    new_x =  unnormalize(new_x, bounds=problem_bounds)
    
    #################### 
    t1 = time.time()
    
    # convert back to matlab/labview format for downloading
    new_x_pandas = pd.DataFrame(new_x.cpu().numpy(), columns=['Qtsc', 'Qag', 'Qpva', 'Qseed', 'Qaa'])
    new_x_pandas['Cond'] = new_x_pandas.index + BATCH_SIZE*exp_counter + initial_sample_size + 1
    new_x_pandas['Repair'] = repair_array
    new_x_pandas['Violation1'] = constraint_array1
    new_x_pandas['Violation2'] = constraint_array2
    new_x_pandas['Time'] =  t1-t0
    new_x_pandas['Candidate Check'] = candidate_check
    new_x_pandas = new_x_pandas[['Cond', 'Qtsc', 'Qag', 'Qpva', 'Qseed', 'Qaa', 'Repair', 'Violation1', 'Violation2', 'Time', 'Candidate Check']] 
        
        
    # Save new flowrate values in csv files before shuffling
    if algo == 'pure':
        output_file = "Flowrates/Flowrates_Algo1_BS_Run" + str(exp_counter+1)
        new_x_pandas.to_csv(f'{output_file}.csv', index=False)
        
    else:
        output_file = "Flowrates/Flowrates_Algo2_BS_Run" + str(exp_counter+1)
        new_x_pandas.to_csv(f'{output_file}.csv', index=False)
        
         # Shuffle flowrate
        input_file = "Flowrates/Flowrates_Algo1_BS_Run" + str(exp_counter+1) + ".csv"            
        dataAlgo1_pandas = pd.read_csv(f"{input_file}", delimiter=',') 
        dataAlgo1_torch = torch.tensor(dataAlgo1_pandas.iloc[:,:].values, **tkwargs)

        input_file = "Flowrates/Flowrates_Algo2_BS_Run" + str(exp_counter+1) + ".csv"            
        dataAlgo2_pandas = pd.read_csv(f"{input_file}", delimiter=',') 
        dataAlgo2_torch = torch.tensor(dataAlgo2_pandas.iloc[:,:].values, **tkwargs)
   
        iOrder = [0,4,1,5,2,6,3,7]
        iSave = 4
        pSave = [0,1,2,3]
        qSave = [0,1,2,3]
            
        for p in multiset_permutations([0,1,2,3]):
            for q in multiset_permutations([0,1,2,3]):
                dataAlgo_torch = torch.vstack([dataAlgo1_torch[p,1:6], dataAlgo2_torch[q,1:6]])
                dataAlgoOrder_torch = dataAlgo_torch[iOrder]

                Test = dataAlgoOrder_torch
                Test[Test <= 4] = 1
                Test[Test > 4] = 0

                # Cumulative sum
                cumsumsave = torch.zeros(2*BATCH_SIZE-1, 1, **tkwargs)
                for i in range(2*BATCH_SIZE-1):
                    cumsum = torch.sum(Test[i:i+2],0)
                    cumsumsave[i] = max(cumsum)

                if(max(cumsumsave) < 2) & (iSave > 1):
                    pSave = p
                    qSave = q
                    iSave = 1
                    print(Test)
                    print(iSave)
                    print(pSave)
                    print(qSave)
                else:
                    # Cumulative sum
                    cumsumsave = torch.zeros(2*BATCH_SIZE-2, 1, **tkwargs)
                    for i in range(2*BATCH_SIZE-2):
                        cumsum = torch.sum(Test[i:i+3],0)
                        cumsumsave[i] = max(cumsum)
                    
                    if(max(cumsumsave) < 3) & (iSave > 2):
                        pSave = p
                        qSave = q
                        iSave = 2
                        print(Test)
                        print(iSave)
                        print(pSave)
                        print(qSave)
                    else:
                        # Cumulative sum
                        cumsumsave = torch.zeros(2*BATCH_SIZE-3, 1, **tkwargs)
                        for i in range(2*BATCH_SIZE-3):
                            cumsum = torch.sum(Test[i:i+4],0)
                            cumsumsave[i] = max(cumsum)

                        if(max(cumsumsave) < 4) & (iSave > 3):
                            pSave = p
                            qSave = q
                            iSave = 3
                            print(Test)
                            print(iSave)
                            print(pSave)
                            print(qSave)
                
        print(pSave)
        print(qSave)
        
        dataAlgo_torch = torch.vstack([dataAlgo1_torch[pSave], dataAlgo2_torch[qSave]])
        dataAlgoOrder_torch = dataAlgo_torch[iOrder]
        new_dataAlgo1_pandas = pd.DataFrame(dataAlgo1_torch[pSave,:].cpu().numpy(), columns=['Cond', 'Qtsc', 'Qag', 'Qpva', 'Qseed', 'Qaa', 'Repair', 'Violation1', 'Violation2', 'Time'])
        new_dataAlgo2_pandas = pd.DataFrame(dataAlgo2_torch[qSave,:].cpu().numpy(), columns=['Cond', 'Qtsc', 'Qag', 'Qpva', 'Qseed', 'Qaa', 'Repair', 'Violation1', 'Violation2', 'Time'])
        
        new_dataAlgo1_pandas['Cond'] = new_dataAlgo1_pandas.index + BATCH_SIZE*exp_counter + initial_sample_size + 1
        new_dataAlgo2_pandas['Cond'] = new_dataAlgo2_pandas.index + BATCH_SIZE*exp_counter + initial_sample_size + 1

        # Save new flowrate values in csv files after shuffling
        output_file = "Flowrates/Flowrates_Algo1_Run" + str(exp_counter+1)
        new_dataAlgo1_pandas.to_csv(f'{output_file}.csv', index=False)

        output_file = "Flowrates/Flowrates_Algo2_Run" + str(exp_counter+1)
        new_dataAlgo2_pandas.to_csv(f'{output_file}.csv', index=False)

    #################### 
    # end of experiment loop
        
    if algo == 'pure':
        print(f"Experiment number {exp_counter} for pure BO , time taken: {t1-t0:>4.2f}s.\nTotal experiments run: {exp_counter}")
        algo = 'hybrid' # switch to next algo

    else:
        print(f"Experiment number {exp_counter} for hybrid BO, time taken: {t1-t0:>4.2f}s.\nTotal experiments run: {exp_counter}")
        algo = 'pure'  # switch to next algo
        exp_counter+=1
    
    print("Sleeping for 5 seconds before next experiment.")
    time.sleep(5) # pause for x sec
    